## Load data

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical

Using Theano backend.


In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [3]:
datapath = '/media/windows-share/EEGs_Guinea-Bissau_np/'

In [4]:
condition = '_10seconds_closed.npy'
X_train = np.load(datapath+'X_train'+condition)
y_train = np.load(datapath+'y_train'+condition)
X_val = np.load(datapath+'X_valid'+condition)
y_val = np.load(datapath+'y_valid'+condition)
X_test = np.load(datapath+'X_test'+condition)
y_test = np.load(datapath+'y_test'+condition)

In [5]:
classlabels = list(set(y_train))
mapclasses = {classlabels[i] : i for i in range(len(classlabels))}
print(mapclasses)

{'Epilepsy': 0, 'Control': 1}


In [6]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_test], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [7]:
y_val_binary

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [8]:
num_classes = y_train_binary.shape[1]
print(num_classes)

2


## Generate models

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam

In [10]:
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))

from mcfly import modelgen, find_architecture

In [11]:
%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 5,
                                  model_type = None)

CPU times: user 4.46 s, sys: 81.1 ms, total: 4.55 s
Wall time: 5.25 s


In [12]:
%%time
for model, params, model_types in models:
    print(params)
    model.summary()

{'lstm_dims': array([64, 65, 35, 45]), 'learning_rate': 0.006520246695352116, 'regularization_rate': 0.0035159397648005524, 'filters': array([86])}
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
reshape_1 (Reshape)                (None, 1, 1280, 14) 0           reshape_input_1[0][0]            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)    (None, 86, 1280, 14)344         reshape_1[0][0]                  
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 86, 1280, 14)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
reshape_2 (Reshape)                (None, 12

## Compare models

In [13]:
%%time
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models,nr_epochs=5,
                                                                                  subsize_set=200,
                                                                                  verbose=True)

Train on 108 samples, validate on 20 samples
Epoch 1/5
108/108 [==============================] - 119s - loss: 0.7261 - acc: 0.5278 - val_loss: 0.7148 - val_acc: 0.5500
Epoch 2/5
 80/108 [=====================>........] - ETA: 38s - loss: 0.7732 - acc: 0.5500

KeyboardInterrupt: 

In [ ]:
%%time
for i in range(len(models)):
    name = str(models[i][1])
    find_architecture.plotTrainingProcess(histories[i], name)

In [ ]:
%%time
import pandas as pd
results = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
results

In [ ]:
results

## Train the best model for real

In [ ]:
best_model_index = np.argmax(val_accuracies)
#best_model_index = 3

In [ ]:
best_model, best_params, best_model_types = models[best_model_index]
print(best_model_index, best_model_types, best_params)

In [ ]:
nr_epochs = 10
datasize = X_train.shape[0]#1000
history = best_model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary))

In [ ]:
find_architecture.plotTrainingProcess(history)

## Test on Testset

In [ ]:
score_test = best_model.evaluate(X_test, y_test_binary, verbose=False)
print('Score of best model: ' + str(score_test))

In [ ]:
best_model.get_config()[0]